In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='http://www.scielo.org.mx/scielo.php?script=sci_issues&pid=2448-7678&lng=en&nrm=iso'
url_root= 'http://www.scielo.org.mx/scielo.php?script=sci_issues&pid=2448-7678&lng=en&nrm=iso'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
box = soup.find('div', class_='content')
volumen=box.findAll('td', width='5%')
volu=box.findAll('a')
vol = [x.get('href')for x in volu]
vol=[urljoin(url_root,i) for i in vol]
vol

In [ ]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='content')
    volumen=box.findAll('td', width='5%')
    volu=box.findAll('a')
    vol = [x.get('href')for x in volu]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Investigación Administrativa'
    a='V. Ciencias Sociales'
    te='Ciencia administrativa'
    ul='http://www.scielo.org.mx/scielo.php?script=sci_serial&pid=2448-7678&lng=en&nrm=iso'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('font', class_='nomodel').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', class_='panel-body').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    urlI=ul
    if urlI:
        content_book['Url Inicial']= urlI
    else :
        content_book['Url Inicial']=None
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', align='left').find('a').get('href')
        content_book['Link Articulo']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link Articulo']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:32]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revist138.csv', index=False)